### Setup

In [1]:
import re
import os
import pandas as pd
path_master = r'C:/Users/endwy/Documents/Columbia MSBA/Spring 2019/E4524 - Analytics in Practice/Data/Csv/'
file_names = list()
for path, subdirs, files in os.walk(path_master):
    for filename in files:
        file_names.append(filename)
df_final = pd.DataFrame()

### All Functions to read/clean CSVs -- produces df_ABR.csv final output

In [ ]:
def convert_1(x):
    if re.search(pattern = r'\w+',string=x):
        return x
    x=x.strip()
    if x[-1]=='%':
        return float(x.replace('%,'))
    if x=='-':
        return 0
    if x[0]=='$':
        return float(x[1:].replace(',','').replace('-','0'))

def ABR_Clean(df):
    """Cleans fully merged df_ABR columns to ensure appropriate dtypes/formats"""
    for col in df.columns:
        df[col]=df[col].apply(convert_1)

def add_season(month):
    springs = ['February','March']
    falls = ['September','October','November','December']
    if month in springs:
        season = "Spring"
    elif month in falls:
        season = "Fall"
    else: 
        season = "Other"
    return season

def merge_cols(df,col1,col2):
    """Merge any two columns in df"""
    for i in range(len(df)):
        df.iloc[i].loc[col1] = (df.iloc[i].loc[col1])+(df.iloc[i].loc[col2])
    del df[col2]
    
def add_time(df_ABR1):
    """Add Month, Year, Season columns based on read-in file name"""
    mon=re.search(r'\w+', file_name)
    df_ABR1['Month']= file_name[mon.start():mon.end()]
    y=re.search(r'\d+', file_name)
    df_ABR1['Year'] = file_name[y.start():y.end()]            
    
def type1(pm,tt,fn,d='.csv'):
    """10 columns, missing 7,9, merge 3,4"""
    with open(pm+tt+fn+d) as fp:
        df_ABR1 = pd.read_csv(fp, 
                              usecols=['0','1','2','3','4','5','6','8','10'],
                              keep_default_na=False)
    df_ABR1.rename(columns={'0':'Drug', '1':'Treated', '2':'Repulls', '3':'Mortalities', '4':'MortalitiesB', 
                        '5':'Treatment Success(%)', '6':'Case Fatality Rate(%)', '8':'$/Hd/Rx','10':'Treatment $'}, inplace=True)
    # Concatenate mortalities column
    merge_cols(df_ABR1,'Mortalities','MortalitiesB')
    add_time(df_ABR1)
    global df_final
    df_final = pd.concat([df_final,df_ABR1])

def type2(pm,tt,fn,d='.csv'):
    """9 columns, missing 6,8"""
    with open(pm+tt+fn+d) as fp:
        df_ABR1 = pd.read_csv(fp,
                              usecols=['0','1','2','3','4','5','7','9'], keep_default_na=False)
    df_ABR1.rename(columns={'0':'Drug', '1':'Treated', '2':'Repulls', '3':'Mortalities', '4':'Treatment Success(%)', 
                            '5':'Case Fatality Rate(%)', '7':'$/Hd/Rx', '9':'Treatment $',}, 
                       inplace=True)
    add_time(df_ABR1)
    global df_final
    df_final = pd.concat([df_final,df_ABR1])

def type3(pm,tt,fn,d='.csv'):
    """base case"""
    with open(pm+tt+fn+d) as fp:
        df_ABR1 = pd.read_csv(fp,
                          skiprows=[0], 
                          usecols=['Unnamed: 1', 'Treated', 'Repulls', 'Mortalities', 'Treatment Success',
                                   'Case Fatality Rate','$/Hd/Rx', 'Treatment $'])
    df_ABR1.rename(columns={'Unnamed: 1': 'Drug', 'Treatment Success':'Treatment Success(%)',
                        'Case Fatality Rate':'Case Fatality Rate(%)'}, inplace=True)
    add_time(df_ABR1)
    global df_final
    df_final = pd.concat([df_final,df_ABR1])

def type4(pm,tt,fn,d='.csv'):
    """11 columns, missing 7,9, concatenate Treatment column"""
    with open(pm+tt+fn+d) as fp:
                df_ABR1 = pd.read_csv(fp,
                         usecols=['0','1','2','3','4','5','6','8','10','11'],
                         keep_default_na=False)
    df_ABR1.rename(columns={'0':'Drug', '1':'Treated', '2':'Repulls', '3':'RepullsB', '4':'Mortalities', 
                        '5':'Treatment Success(%)', '6':'Case Fatality Rate(%)', '8':'$/Hd/Rx',
                        '10':'Treatment $','11':'Treatment $B'}, inplace=True)
    # Concatenate Treatment $ column and Repulls column
    merge_cols(df_ABR1,'Treatment $','Treatment $B')
    merge_cols(df_ABR1,'Repulls','RepullsB')
    add_time(df_ABR1)
    global df_final
    df_final = pd.concat([df_final,df_ABR1])

def type5(pm,tt,fn,d='.csv'):
    """10 column, missing 6,8, merge 9,10"""
    with open(pm+tt+fn+d) as fp:
                df_ABR1 = pd.read_csv(fp,
                         usecols=['0','1','2','3','4','5','7','9','10'],
                         keep_default_na=False)
    df_ABR1.rename(columns={'0':'Drug', '1':'Treated', '2':'Repulls', '3':'Mortalities', '4':'Treatment Success(%)', 
                        '5':'Case Fatality Rate(%)', '7':'$/Hd/Rx', 
                        '9':'Treatment $','10':'Treatment $B'}, inplace=True)
    # Concatenate Treatment $ column
    merge_cols(df_ABR1,'Treatment $','Treatment $B')
    add_time(df_ABR1)
    global df_final
    df_final = pd.concat([df_final,df_ABR1])

def type6(pm,tt,fn,d='.csv'):
    """11 column, concatenate Treatment column"""
    with open(path_master+title+file_num+'.csv') as fp:
                df_ABR1 = pd.read_csv(fp,
                         usecols=['0','1','3','4','5','6','7','8','9','10','11'],
                         keep_default_na=False)
    df_ABR1.rename(columns={'0':'Drug', '1':'Treated', '3':'Repulls', '4':'Mortalities','5':'MortalitiesB','6':'Treatment Success(%)', 
                        '7':'Case Fatality Rate(%)', '8':'$/Hd/Rx', '9':'$/Hd/RxB',
                        '10':'Treatment $','11':'Treatment $B'}, inplace=True)
    # Concatenate Treatment $ column
    merge_cols(df_ABR1,'Mortalities','MortalitiesB')
    merge_cols(df_ABR1,'$/Hd/Rx','$/Hd/RxB')
    merge_cols(df_ABR1,'Treatment $','Treatment $B')
    df_ABR1['Treated']=df_ABR1['Treated'].apply(lambda x: x.replace(' ',''))
    add_time(df_ABR1)
    global df_final
    df_final = pd.concat([df_final,df_ABR1])

def type7(pm,tt,fn,d='.csv'):
    """11 columns, missing 8,10"""
    with open(path_master+title+file_num+'.csv') as fp:
                df_ABR1 = pd.read_csv(fp,
                         usecols=['0','1','2','4','6','7','9','11'], keep_default_na=False)
    df_ABR1.rename(columns={'0':'Drug', '1':'Treated', '2':'Repulls', '4':'Mortalities', '6':'Treatment Success(%)', 
                        '7':'Case Fatality Rate(%)', '9':'$/Hd/Rx', '11':'Treatment $',
                        }, inplace=True)
    add_time(df_ABR1)
    global df_final
    df_final = pd.concat([df_final,df_ABR1])

def type8(pm,tt,fn,d='.csv'):
    """10 columns, missing 8,9, concatenate Mortalities column"""
    with open(pm+tt+fn+d) as fp:
        df_ABR1 = pd.read_csv(fp, 
                              usecols=['0','1','2','3','4','5','6','7','10'],
                              keep_default_na=False)
    df_ABR1.rename(columns={'0':'Drug', '1':'Treated', '2':'Repulls', '3':'Mortalities', '4':'MortalitiesB', 
                        '5':'Treatment Success(%)', '6':'Case Fatality Rate(%)', '7':'$/Hd/Rx','10':'Treatment $'},
                   inplace=True)
    # Concatenate mortalities column
    merge_cols(df_ABR1,'Mortalities','MortalitiesB')
    add_time(df_ABR1)
    global df_final
    df_final = pd.concat([df_final,df_ABR1])

def type9(pm,tt,fn,d='.csv'):
    """13 columns, missing 10,12"""
    with open(pm+tt+fn+d) as fp:
        df_ABR1 = pd.read_csv(fp, 
                              usecols=['0','1','2','3','5','6','8','9','11','13'],
                              keep_default_na=False)
    df_ABR1.rename(columns={'0':'Drug', '1':'Treated','2':'Repulls','3':'RepullsB','5':'Mortalities','6':'MortalitiesB',
                            '8':'Treatment Success(%)','9':'Case Fatality Rate(%)','11':'$/Hd/Rx','13':'Treatment $'},
                   inplace=True)
    # Concatenate Repulls & Mortalities columns
    merge_cols(df_ABR1,'Mortalities','MortalitiesB')
    merge_cols(df_ABR1,'Repulls','RepullsB')
    add_time(df_ABR1)
    global df_final
    df_final = pd.concat([df_final,df_ABR1])

def type10(pm,tt,fn,d='.csv'):
    """9 columns, missing 8, merge 3,4"""
    with open(pm+tt+fn+d) as fp:
        df_ABR1 = pd.read_csv(fp, 
                              usecols=['0','1','2','3','4','5','6','7','9'],
                              keep_default_na=False)
    df_ABR1.rename(columns={'0':'Drug', '1':'Treated', '2':'Repulls', '3':'Mortalities', '4':'MortalitiesB', 
                        '5':'Treatment Success(%)', '6':'Case Fatality Rate(%)', '7':'$/Hd/Rx','9':'Treatment $'}, 
                   inplace=True)
    # Concatenate mortalities column
    merge_cols(df_ABR1,'Mortalities','MortalitiesB')
    add_time(df_ABR1)
    global df_final
    df_final = pd.concat([df_final,df_ABR1])
    
def type11(pm,tt,fn,d='.csv'):
    """11 columns, missing 3,8,10 merge 4,5"""
    with open(path_master+title+file_num+'.csv') as fp:
                df_ABR1 = pd.read_csv(fp,
                         usecols=['0','1','2','4','5','6','7','9','11'], keep_default_na=False)
    df_ABR1.rename(columns={'0':'Drug', '1':'Treated', '2':'Repulls', '4':'Mortalities', '5':'MortalitiesB',
                            '6':'Treatment Success(%)', '7':'Case Fatality Rate(%)', '9':'$/Hd/Rx', '11':'Treatment $',
                        }, inplace=True)
     # Concatenate mortalities column
    merge_cols(df_ABR1,'Mortalities','MortalitiesB')
    add_time(df_ABR1)
    global df_final
    df_final = pd.concat([df_final,df_ABR1])
    
def type12(pm,tt,fn,d='.csv'):
    """11 columns, missing 8,10, merge 2,3 and 4,5"""
    with open(path_master+title+file_num+'.csv') as fp:
                df_ABR1 = pd.read_csv(fp,
                         usecols=['0','1','2','3','4','5','6','7','9','11'], keep_default_na=False)
    df_ABR1.rename(columns={'0':'Drug', '1':'Treated', '2':'Repulls', '3':'RepullsB','4':'Mortalities','5':'MortalitiesB',
                            '6':'Treatment Success(%)','7':'Case Fatality Rate(%)', '9':'$/Hd/Rx', '11':'Treatment $',
                        }, inplace=True)
    # Concatenate Repulls & Mortalities columns
    merge_cols(df_ABR1,'Mortalities','MortalitiesB')
    merge_cols(df_ABR1,'Repulls','RepullsB')
    add_time(df_ABR1)
    global df_final
    df_final = pd.concat([df_final,df_ABR1])
    
def type13(pm,tt,fn,d='.csv'):
    """11 column, missing 7,9, merge 2,3 and 10,11"""
    with open(path_master+title+file_num+'.csv') as fp:
                df_ABR1 = pd.read_csv(fp,
                         usecols=['0','1','2','3','4','5','6','8','10','11'],
                         keep_default_na=False)
    df_ABR1.rename(columns={'0':'Drug', '1':'Treated','2':'Repulls', '3':'RepullsB', '4':'Mortalities',
                            '5':'Treatment Success(%)', '6':'Case Fatality Rate(%)', 
                            '8':'$/Hd/Rx','10':'Treatment $','11':'Treatment $B'}, inplace=True)
    # Concatenate Treatment $ column
    merge_cols(df_ABR1,'Repulls','RepullsB')
    merge_cols(df_ABR1,'Treatment $','Treatment $B')
    df_ABR1['Treated']=df_ABR1['Treated'].apply(lambda x: x.replace(' ',''))
    add_time(df_ABR1)
    global df_final
    df_final = pd.concat([df_final,df_ABR1])
    
def type14(pm,tt,fn,d='.csv'):
    """9 columns, missing 7,8"""
    with open(pm+tt+fn+d) as fp:
        df_ABR1 = pd.read_csv(fp,
                              usecols=['0','1','2','3','4','5','6','9'], keep_default_na=False)
    df_ABR1.rename(columns={'0':'Drug', '1':'Treated', '2':'Repulls', '3':'Mortalities', '4':'Treatment Success(%)', 
                            '5':'Case Fatality Rate(%)', '6':'$/Hd/Rx', '9':'Treatment $',}, 
                       inplace=True)
    add_time(df_ABR1)
    global df_final
    df_final = pd.concat([df_final,df_ABR1])
    


### Use functions to clean each month's CSVs

In [ ]:
# JANUARY 2017 & 2018 (note: no 2017 file)
for file_name in file_names:
    pattern = r'January\s+\d+\s+\-\s+Antibiotic by Risk'
    title = file_name.split("Risk")[0]+str("Risk")
    if re.search(pattern=pattern, string=file_name):
        file_num=re.findall(r'\d+', file_name)[-1]
        # cleaning for 2018
        if file_name.split()[1] == '2018':
            if file_num == '1':
                type3(path_master,title,file_num)
            elif file_num == '2':
                type4(path_master,title,file_num)
            elif file_num == '3' or file_num == '5' or file_num == '8'or file_num == '10':
                type1(path_master,title,file_num)
            elif file_num == '4':
                type5(path_master,title,file_num)
            elif file_num == '6' or file_num == '7'or file_num == '9':
                type2(path_master,title,file_num)    

# FEBRUARY 2017 & 2018
for file_name in file_names:
    pattern = r'February\s+\d+\s+\-\s+Antibiotic by Risk'
    title = file_name.split("Risk")[0]+str("Risk")
    if re.search(pattern=pattern, string=file_name):
        file_num=re.findall(r'\d+', file_name)[-1]
        # cleaning for 2017
        if file_name.split()[1] == '2017':
            if file_num == '1':
                type3(path_master,title,file_num)
            elif file_num == '2':
                type4(path_master,title,file_num)
            elif file_num == '3' or file_num == '5'or file_num == '8':
                type1(path_master,title,file_num)
            elif file_num == '4'or file_num == '9':
                type2(path_master,title,file_num)
            elif file_num == '6':
                type5(path_master,title,file_num)
            elif file_num == '7'or file_num == '10':
                type7(path_master,title,file_num)
        # cleaning for 2018
        elif file_name.split()[1] == '2018':
            if file_num == '1':
                type3(path_master,title,file_num)
            elif file_num == '2':
                type4(path_master,title,file_num)
            elif file_num == '3' or file_num == '10':
                type1(path_master,title,file_num)
            elif file_num == '4':
                type5(path_master,title,file_num)
            elif file_num == '5':
                type8(path_master,title,file_num)
            elif file_num == '6' or file_num == '7' or file_num == '9':
                type2(path_master,title,file_num)
            elif file_num == '8':
                type7(path_master,title,file_num)
        
# MARCH 2017 & 2018 (note: no 2017 file)
for file_name in file_names:
    pattern = r'March\s+\d+\s+\-\s+Antibiotic by Risk'
    title = file_name.split("Risk")[0]+str("Risk")
    if re.search(pattern=pattern, string=file_name):
        file_num=re.findall(r'\d+', file_name)[-1]
        # cleaning for 2018
        if file_name.split()[1] == '2018':
            if file_num == '1':
                type3(path_master,title,file_num)
            elif file_num == '2':
                type4(path_master,title,file_num)
            elif file_num == '3':
                type7(path_master,title,file_num)
            elif file_num == '4' or file_num == '5'or file_num == '6'or file_num == '7'or file_num == '8':
                type1(path_master,title,file_num)
            elif file_num == '9':
                type2(path_master,title,file_num)
            elif file_num == '10':
                type9(path_master,title,file_num)
    
# APRIL 2017 & 2018
for file_name in file_names:
    pattern = r'April\s+\d+\s+\-\s+Antibiotic by Risk'
    title = file_name.split("Risk")[0]+str("Risk")
    if re.search(pattern=pattern, string=file_name):
        file_num=re.findall(r'\d+', file_name)[-1]
        # cleaning for 2017
        if file_name.split()[1] == '2017':
            if file_num == '3' or file_num == '5' or file_num == '8':
                type1(path_master,title,file_num)
            elif file_num == '4' or file_num == '9':
                type2(path_master,title,file_num)
            elif file_num == '1':
                type3(path_master,title,file_num)
            elif file_num == '2':
                type4(path_master,title,file_num)
            elif file_num == '6':
                type5(path_master,title,file_num)
            elif file_num == '7':
                type6(path_master,title,file_num)
            elif file_num == '10':
                type7(path_master,title,file_num)
        # cleaning for 2018
        elif file_name.split()[1] == '2018':
            if file_num == '1':
                type3(path_master,title,file_num)
            elif file_num == '2':
                type5(path_master,title,file_num)
            elif file_num == '3' or file_num=='4' or file_num=='6' or file_num=='7' or file_num=='10':
                type1(path_master,title,file_num)  
            elif file_num == '3' or file_num=='5' or file_num=='9':
                type2(path_master,title,file_num)
            elif file_num == '8':
                type7(path_master,title,file_num)
                
# MAY 2017 & 2018 (note: no 2017 file)
for file_name in file_names:
    pattern = r'May\s+\d+\s+\-\s+Antibiotic by Risk'
    title = file_name.split("Risk")[0]+str("Risk")
    if re.search(pattern=pattern, string=file_name):
        file_num=re.findall(r'\d+', file_name)[-1]
        # cleaning for 2018
        if file_name.split()[1] == '2018':
            if file_num == '1':
                type3(path_master,title,file_num)
            elif file_num == '2':
                type5(path_master,title,file_num)
            elif file_num == '3' or file_num == '10':
                type1(path_master,title,file_num)
            elif file_num == '4' or file_num == '5' or file_num == '6'or file_num == '7'or file_num == '8'or file_num == '9':
                type2(path_master,title,file_num)
                
# JUNE 2017 & 2018 (note: no 2017 file)
for file_name in file_names:
    pattern = r'June\s+\d+\s+\-\s+Antibiotic by Risk'
    title = file_name.split("Risk")[0]+str("Risk")
    if re.search(pattern=pattern, string=file_name):
        file_num=re.findall(r'\d+', file_name)[-1]
        # cleaning for 2018
        if file_name.split()[1] == '2018':
            if file_num == '1':
                type3(path_master,title,file_num)
            elif file_num == '2':
                type7(path_master,title,file_num)
            elif file_num == '3' or file_num == '4' or file_num == '10':
                type1(path_master,title,file_num)
            elif file_num == '5' or file_num == '6' or file_num == '7'or file_num == '8':
                type2(path_master,title,file_num)
            elif file_num == '9':
                type5(path_master,title,file_num)
                
# JULY 2017 & 2018 (note: no 2017 file)
for file_name in file_names:
    pattern = r'July\s+\d+\s+\-\s+Antibiotic by Risk'
    title = file_name.split("Risk")[0]+str("Risk")
    if re.search(pattern=pattern, string=file_name):
        file_num=re.findall(r'\d+', file_name)[-1]
        # cleaning for 2018
        if file_name.split()[1] == '2018':
            if file_num == '1':
                type3(path_master,title,file_num)
            elif file_num == '2' or file_num == '4' or file_num == '5' or file_num == '6' or file_num == '7' or file_num == '8':
                type2(path_master,title,file_num)
            elif file_num == '3':
                type1(path_master,title,file_num)
            elif file_num == '9':
                type5(path_master,title,file_num)
            elif file_num == '10':
                type12(path_master,title,file_num)
                
# AUGUST 2017 & 2018
for file_name in file_names:
    pattern = r'August\s+\d+\s+\-\s+Antibiotic by Risk'
    title = file_name.split("Risk")[0]+str("Risk")
    if re.search(pattern=pattern, string=file_name):
        file_num=re.findall(r'\d+', file_name)[-1]
        # cleaning for 2017
        if file_name.split()[1] == '2017':
            if file_num == '1':
                type3(path_master,title,file_num)
            elif file_num == '2' or file_num == '4' or file_num == '7' or file_num == '8':
                type2(path_master,title,file_num)
            elif file_num == '3' or file_num == '5':
                type1(path_master,title,file_num)
            elif file_num == '6':
                type5(path_master,title,file_num)
            elif file_num == '9':
                type4(path_master,title,file_num)
            elif file_num == '10':
                type7(path_master,title,file_num)
        # cleaning for 2018
        if file_name.split()[1] == '2018':
            if file_num == '1':
                type3(path_master,title,file_num)
            elif file_num == '2' or file_num == '4' or file_num == '5' or file_num == '7' or file_num == '9':
                type2(path_master,title,file_num)
            elif file_num == '3':
                type10(path_master,title,file_num)
            elif file_num == '6' or file_num == '8':
                type1(path_master,title,file_num)
            elif file_num == '10':
                type12(path_master,title,file_num)
                
# SEPTEMBER 2017 & 2018 (note: no 2017 file)
for file_name in file_names:
    pattern = r'September\s+\d+\s+\-\s+Antibiotic by Risk'
    title = file_name.split("Risk")[0]+str("Risk")
    if re.search(pattern=pattern, string=file_name):
        file_num=re.findall(r'\d+', file_name)[-1]
        # cleaning for 2018
        if file_name.split()[1] == '2018':
            if file_num == '1':
                type3(path_master,title,file_num)
            elif file_num == '2' or file_num == '3' or file_num == '5' or file_num == '8' or file_num == '10':
                type1(path_master,title,file_num)
            elif file_num == '4':
                type5(path_master,title,file_num)
            elif file_num == '6' or file_num == '7' or file_num == '9':
                type2(path_master,title,file_num)
                
# OCTOBER 2017 & 2018 (note: no 2017 file)
for file_name in file_names:
    pattern = r'October\s+\d+\s+\-\s+Antibiotic by Risk'
    title = file_name.split("Risk")[0]+str("Risk")
    if re.search(pattern=pattern, string=file_name):
        file_num=re.findall(r'\d+', file_name)[-1]
        # cleaning for 2018
        if file_name.split()[1] == '2018':
            if file_num == '1':
                type3(path_master,title,file_num)
            elif file_num == '2':
                type12(path_master,title,file_num)
            elif file_num == '3' or file_num == '8' or file_num == '10':
                type1(path_master,title,file_num)
            elif file_num == '4':
                type13(path_master,title,file_num)
            elif file_num == '5':
                type14(path_master,title,file_num)
            elif file_num == '6' or file_num == '7' or file_num == '9':
                type2(path_master,title,file_num)
            
# NOVEMBER 2017 & 2018 (note: no 2017 or 2018 file)

# DECEMBER 2017 & 2018 (note: no 2018 file)
for file_name in file_names:
    pattern = r'December\s+\d+\s+\-\s+Antibiotic by Risk'
    title = file_name.split("Risk")[0]+str("Risk")
    if re.search(pattern=pattern, string=file_name):
        file_num=re.findall(r'\d+', file_name)[-1]
        # cleaning for 2017
        if file_name.split()[1] == '2017':
            if file_num == '1':
                type3(path_master,title,file_num)
            elif file_num == '2' or file_num == '4':
                type5(path_master,title,file_num)
            elif file_num == '3' or file_num == '4' or file_num == '8' or file_num == '10':
                type1(path_master,title,file_num)
            elif file_num == '6' or file_num == '7' or file_num == '9':
                type2(path_master,title,file_num)

# Remove percent signs from columns and convery str -> float
df_final['Treatment Success(%)'] = df_final['Treatment Success(%)'].apply(lambda x: float(x[:-1]))
df_final['Case Fatality Rate(%)'] = df_final['Case Fatality Rate(%)'].apply(lambda x: float(x[:-1]))

# add season column
df_final['Season'] = df_final['Month'].apply(add_season)

# Save out df_ABR
import csv
df_final.to_csv(path_master+"/df_ABR.csv")

### Can just open full file below to work on cleaning full df

In [2]:
import pandas as pd
with open(path_master+'df_ABR.csv') as fp:
    df_final = pd.read_csv(fp)
del df_final['Unnamed: 0']

In [4]:
def conv(x):
    x = str(x)
    x = x.strip().replace(',','')
    x = x.replace(' ','')
    if x=='-': 
        return 0
    else:
        return x
ignore_names = ['Drug','Treatment Success(%)','Case Fatality Rate(%)','Month','Year','Season']

In [5]:
for col in df_final.columns:
    if col not in ignore_names:
        df_final[col] = df_final[col].apply(conv)

In [6]:
df_final.head(15)

,Drug,Treated,Repulls,Mortalities,Treatment Success(%),Case Fatality Rate(%),$/Hd/Rx,Treatment $,Month,Year,Season
0,Low Risk,38732,2695,1455,93.04,3.76,$25.88,$1002416.00,January,2018,Other
1,300 PRO LA,0,0,0,0.00,0.00,$-,$-,January,2018,Other
2,A180,0,0,0,0.00,0.00,$-,$-,January,2018,Other
3,A180 As 2nd Rx,0,0,0,0.00,0.00,$-,$-,January,2018,Other
4,Adspec,0,0,0,0.00,0.00,$-,$-,January,2018,Other
5,Advocin,142,23,14,83.80,9.86,$36.13,$5130.82,January,2018,Other
6,Advocin As 2nd Rx,716,1,118,99.86,16.48,$28.34,$20291.44,January,2018,Other
7,Albon Bolus,0,0,0,0.00,0.00,$-,$-,January,2018,Other
8,Baytril,4,4,1,0.00,25.00,$22.88,$91.52,January,2018,Other
9,Baytril As 2nd Rx,1128,1,112,99.91,9.93,$22.40,$25267.20,January,2018,Other


In [ ]:
# Clean df_final
for i in range(len(df_final)):
    for j in ['Treated','Repulls','Mortalities']:
        if df_final.iloc[i].loc[j] == '-':
            df_final.iloc[i].loc[j] = 0
        else:
            m = df_final.iloc[i].loc[j]
            m = m.replace(' ','')
            df_final.iloc[i].loc[j] = m.replace(',','')

# Change str -> int for numerical columns
# df_final['Treated'] = df_final['Treated'].apply(lambda x: int(x))
# df_final['Repulls'] = df_final['Repulls'].apply(lambda x: int(x))
# df_final['Mortalities'] = df_final['Mortalities'].apply(lambda x: int(x))

df_final['Treated'] = df_final['Treated'].convert_objects(convert_numeric=True)
df_final['Repulls'] = df_final['Repulls'].convert_objects(convert_numeric=True)
df_final['Mortalities'] = df_final['Mortalities'].convert_objects(convert_numeric=True)

# Remove dollar signs from columns
df_final['Treatment $'] = df_final['Treatment $'].apply(lambda x: x[1:])
df_final['$/Hd/Rx'] = df_final['$/Hd/Rx'].apply(lambda x: x[1:])

# Remove dashes from columns
for i in range(len(df_final)):
    for j in ['$/Hd/Rx','Treatment $']:
        if df_final.iloc[i].loc[j] == '-':
            df_final.iloc[i].loc[j] = 0.0
        else:
            m = df_final.iloc[i].loc[j]
            df_final.iloc[i].loc[j] = m.replace(',','')
            m = m.replace(' ','')
            df_final.iloc[i].loc[j] = m.replace(',','')

# Change str -> float for numerical columns
df_final['Treatment $'] = df_final['Treatment $'].convert_objects(convert_numeric=True)
df_final['$/Hd/Rx'] = df_final['$/Hd/Rx'].convert_objects(convert_numeric=True)


In [ ]:
# attempt to clean df_final with functions

def convert_1(x):
    if re.search(pattern = r'\w+',string=x):
        return x
    x=x.strip()
    if x[-1]=='%':
        return float(x.replace('%,'))
    if x=='-':
        return 0
    if x[0]=='$':
        return float(x[1:].replace(',','').replace('-','0'))

def ABR_Clean(df):
    """Cleans fully merged df_ABR columns to ensure appropriate dtypes/formats"""
    for col in df.columns:
        df[col]=df[col].apply(convert_1)
    
# Run this below to convert to .py file before pushing to GitHub
get_ipython().system('jupyter nbconvert --to script Antibiotics_by_Risk.ipynb')